Question 12: Imagine that your model has been in deployed state for a while and has been making predictions. Extend your model deployment application to collect feedback (correct response value) on the predictions made by the model. 
Outline and demonstrate your model refresh strategy. How will you incorporate the feedback so as to retrain and improve the model with time? 

### SKlearn partial fit (http://scikit-learn.org/stable/modules/scaling_strategies.html)

In [ ]:
# import numpy as np
# from sklearn import linear_model
 
# X = np.array([[-1, -1], [-2, -1], [1, 1], [2, 1]])
# Y = np.array([1, 1, 2, 2])
 
# clf = linear_model.SGDClassifier()
# clf.fit(X, Y) # Train the model for the first time.
 
# X_new = np.array([[-1.2, -0.4], [2, 1.9], [-1, 1], [1, 2]])
# Y_new = np.array([2, 1, 1, 2])
 
# clf.partial_fit(X_new, Y_new) # Update the model.

### Offline (retrain on the entire dataset)

### Reinforcement Learning

### Online learning
1. Save new training data as you receive it. For example, if you are receiving updated prices of houses on the market, save that information to a database.
2. When you have enough new data, test its accuracy against your machine learning model.
3. If you see the accuracy of your model degrading over time, use the new data, or a combination of the new data and old training data to build and deploy a new model.

## Re-training 1
1. Split Available dataset into 3 part {df(80):new(20)} -> {train(80):test(20)}
2. Receive New-dataset CSV file (Flask Jinga)
3. Train Model using Train-Test & calculate 'Accuracy before Re-training'
4. Add New data and re-split into {Train:80, Test:20} & calculate 'Accuracy after Re-training'
5. Print Both Accuracy (Flask Jinga)

### 1.Split Available dataset into 3 part {df(80):new(20)} -> {train(80):test(20)}

In [47]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

data = pd.read_csv('DC_Properties.csv')

msk = np.random.rand(len(data)) < 0.8
df = data[msk]
new = data[~msk]

new.to_csv('new.csv', index=False)
df.to_csv('old.csv', index=False)

c:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (18,19,20,21,22,23,24,33,34,35,37,43,45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Training model

In [48]:
removable_features = ['SOURCE', 'CENSUS_TRACT', 'CENSUS_BLOCK', 'SQUARE', 'USECODE', 'Unnamed: 0', 'X', 'Y']
df = df.drop(removable_features, axis=1)

df['SALEDATE'] = pd.to_datetime(df['SALEDATE'])
df['SALE_YR'] = df['SALEDATE'].dt.year; df['SALE_MONTH'] = df['SALEDATE'].dt.month
df = df.drop('SALEDATE', axis=1)
df = df.drop('GIS_LAST_MOD_DTTM', axis=1)
df = df[~pd.isnull(df['PRICE'])]
df = df.drop(df[df.PRICE>2000000].index)

df = df.loc[:, df.isnull().sum() < 0.4*df.shape[0]]
df = df.drop('ASSESSMENT_SUBNBHD', axis=1)
df = df.dropna(how='any')
df = df.drop(['LATITUDE','LONGITUDE','ZIPCODE'], axis=1)

df = df[df.FIREPLACES<=11]
df = df[df.AYB>=1825]
df = df[df.EYB>=1900]
df = df[df.LANDAREA<=50000]

df = pd.concat([df, pd.get_dummies(df[['AC','QUALIFIED','WARD','QUADRANT']])], axis=1)
df = df.drop(['AC','QUALIFIED','WARD','QUADRANT'], axis=1)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['HEAT']=le.fit_transform(df['HEAT'])
df['ASSESSMENT_NBHD']=le.fit_transform(df['ASSESSMENT_NBHD'])

df["PRICE"] = np.log1p(df["PRICE"])
df = df.drop(df[df.PRICE<10].index)
#y = df['PRICE']; X = df.drop('PRICE', axis=1)

msk = np.random.rand(len(df)) < 0.8
train = df[msk]
y_train = train['PRICE']; X_train = train.drop('PRICE', axis=1)

test = df[~msk]
test.to_csv('test_data.csv', index=False)
#X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20)

from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor(random_state=42)
forest_reg.fit(X_train, y_train)

from sklearn.externals import joblib
# now you can save it to a file
joblib.dump(forest_reg, 'oldmodel.pkl')

['oldmodel.pkl']

In [34]:
# '''Runk in Flask python script'''
# predictions = forest_reg.predict(X_test)
# from sklearn import metrics
# mae_before = metrics.mean_absolute_error(y_test, predictions)
# mse_before = metrics.mean_squared_error(y_test, predictions)
# rmse_before = np.sqrt(metrics.mean_squared_error(y_test, predictions))
# rsq_before = metrics.r2_score(y_test, predictions)

### Add new data and re-train

In [35]:
# old_data = pd.read_csv('old.csv')
# new_data = pd.read_csv('new.csv')
# df = pd.concat([old_data, new_data])

# removable_features = ['SOURCE', 'CENSUS_TRACT', 'CENSUS_BLOCK', 'SQUARE', 'USECODE', 'Unnamed: 0', 'X', 'Y']
# df = df.drop(removable_features, axis=1)

# df['SALEDATE'] = pd.to_datetime(df['SALEDATE'])
# df['SALE_YR'] = df['SALEDATE'].dt.year; df['SALE_MONTH'] = df['SALEDATE'].dt.month
# df = df.drop('SALEDATE', axis=1)
# df = df.drop('GIS_LAST_MOD_DTTM', axis=1)
# df = df[~pd.isnull(df['PRICE'])]
# df = df.drop(df[df.PRICE>2000000].index)

# df = df.loc[:, df.isnull().sum() < 0.4*df.shape[0]]
# df = df.drop('ASSESSMENT_SUBNBHD', axis=1)
# df = df.dropna(how='any')
# df = df.drop(['LATITUDE','LONGITUDE','ZIPCODE'], axis=1)

# df = df[df.FIREPLACES<=11]
# df = df[df.AYB>=1825]
# df = df[df.EYB>=1900]
# df = df[df.LANDAREA<=50000]

# df = pd.concat([df, pd.get_dummies(df[['AC','QUALIFIED','WARD','QUADRANT']])], axis=1)
# df = df.drop(['AC','QUALIFIED','WARD','QUADRANT'], axis=1)
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# df['HEAT']=le.fit_transform(df['HEAT'])
# df['ASSESSMENT_NBHD']=le.fit_transform(df['ASSESSMENT_NBHD'])

# df["PRICE"] = np.log1p(df["PRICE"])
# df = df.drop(df[df.PRICE<10].index)
# y = df['PRICE']; X = df.drop('PRICE', axis=1)

# X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20)
# from sklearn.ensemble import RandomForestRegressor
# forest_reg = RandomForestRegressor(random_state=42)
# forest_reg.fit(X_train, y_train)
# predictions = forest_reg.predict(X_test)

# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# from sklearn import metrics
# mae_after = metrics.mean_absolute_error(y_test, predictions)
# mse_after = metrics.mean_squared_error(y_test, predictions)
# rmse_after = np.sqrt(metrics.mean_squared_error(y_test, predictions))
# rsq_after = metrics.r2_score(y_test, predictions)

In [49]:
new_data = pd.read_csv('new.csv')
test_data = pd.read_csv('test_data.csv')

testY_oldmodel = test_data['PRICE'].values
testX_oldmodel = test_data.drop('PRICE', axis=1)

old_model = joblib.load('oldmodel.pkl')
predictions = old_model.predict(testX_oldmodel)
from sklearn import metrics
mae_before = metrics.mean_absolute_error(testY_oldmodel, predictions)
mse_before = metrics.mean_squared_error(testY_oldmodel, predictions)
rmse_before = np.sqrt(metrics.mean_squared_error(testY_oldmodel, predictions))
rsq_before = metrics.r2_score(testY_oldmodel, predictions)

old_data = pd.read_csv('old.csv')
#new_data = pd.read_csv(request.files.get('file')) #From user (through web-interface)
df = pd.concat([old_data, new_data])

removable_features = ['SOURCE', 'CENSUS_TRACT', 'CENSUS_BLOCK', 'SQUARE', 'USECODE', 'Unnamed: 0', 'X', 'Y']
df = df.drop(removable_features, axis=1)

df['SALEDATE'] = pd.to_datetime(df['SALEDATE'])
df['SALE_YR'] = df['SALEDATE'].dt.year; df['SALE_MONTH'] = df['SALEDATE'].dt.month
df = df.drop('SALEDATE', axis=1)
df = df.drop('GIS_LAST_MOD_DTTM', axis=1)
df = df[~pd.isnull(df['PRICE'])]
df = df.drop(df[df.PRICE>2000000].index)

df = df.loc[:, df.isnull().sum() < 0.4*df.shape[0]]
df = df.drop('ASSESSMENT_SUBNBHD', axis=1)
df = df.dropna(how='any')
df = df.drop(['LATITUDE','LONGITUDE','ZIPCODE'], axis=1)

df = df[df.FIREPLACES<=11]
df = df[df.AYB>=1825]
df = df[df.EYB>=1900]
df = df[df.LANDAREA<=50000]

df = pd.concat([df, pd.get_dummies(df[['AC','QUALIFIED','WARD','QUADRANT']])], axis=1)
df = df.drop(['AC','QUALIFIED','WARD','QUADRANT'], axis=1)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['HEAT']=le.fit_transform(df['HEAT'])
df['ASSESSMENT_NBHD']=le.fit_transform(df['ASSESSMENT_NBHD'])

df["PRICE"] = np.log1p(df["PRICE"])
df = df.drop(df[df.PRICE<10].index)
y = df['PRICE']; X = df.drop('PRICE', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20)
from sklearn.ensemble import RandomForestRegressor
forest_reg_new = RandomForestRegressor(random_state=42)
forest_reg_new.fit(X_train, y_train)
predictions = forest_reg_new.predict(X_test)

from sklearn import metrics
mae_after = metrics.mean_absolute_error(y_test, predictions)
mse_after = metrics.mean_squared_error(y_test, predictions)
rmse_after = np.sqrt(metrics.mean_squared_error(y_test, predictions))
rsq_after = metrics.r2_score(y_test, predictions)

c:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
c:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (18,19,20,21,22,23,24,33,34,35,37,43,45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
